<a href="https://colab.research.google.com/github/phfgomes1969/PUC-RJ-MVP2/blob/main/MVP_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import shutil
import glob
import numpy as np
import pandas as pd
import zipfile
import requests
import shutil


import matplotlib.pyplot as plt
import cv2
#from tqdm import tqdm

!pip install ultralytics -q

from ultralytics import YOLO


In [ ]:
diretorios_para_apagar = [
    '/content/dataset/images/train',
    '/content/dataset/images/val',
    '/content/dataset/labels/train',
    '/content/dataset/labels/val',
    '/content/dataset',
    '/content/runs',
    '/content/Fotos_Teste',
    '/content/dataset',
    '/content/treinado']


for diretorio in diretorios_para_apagar:
    if os.path.exists(diretorio):  # Verifica se o diretório existe
        try:
            shutil.rmtree(diretorio)
            print(f"Diretório '{diretorio}' apagado com sucesso.")
        except OSError as e:
            print(f"Erro ao apagar o diretório '{diretorio}': {e}")
    else:
        print(f"Diretório '{diretorio}' não encontrado.")

Diretório '/content/dataset/images/train' apagado com sucesso.
Diretório '/content/dataset/images/val' apagado com sucesso.
Diretório '/content/dataset/labels/train' apagado com sucesso.
Diretório '/content/dataset/labels/val' apagado com sucesso.
Diretório '/content/dataset' apagado com sucesso.
Diretório '/content/runs' não encontrado.
Diretório '/content/Fotos_Teste' não encontrado.
Diretório '/content/dataset' não encontrado.
Diretório '/content/treinado' não encontrado.


In [ ]:
# Busca os arquivos que compoem o dataset no GitHub
# Tive que quebrar em vários pedaços poir limitação do GitHub

repo_url = 'https://api.github.com/repos/phfgomes1969/PUC-RJ-MVP2/contents/'

# Pasta de destino no Google Colab
destino = '/content/dataset'

# Cria a pasta de destino se ela não existir
os.makedirs(destino, exist_ok=True)

# Obtém a lista de arquivos do repositório em formato JSON
resposta = requests.get(repo_url)
resposta.raise_for_status()
conteudo = resposta.json()

# Filtra os arquivos com extensão .zip
arquivos_zip = [arquivo['name'] for arquivo in conteudo if arquivo['name'].endswith('.zip')]

# Baixa e extrai cada arquivo .zip
for nome_arquivo in arquivos_zip:
    # Constrói a URL completa do arquivo
    url_arquivo = f"https://raw.githubusercontent.com/phfgomes1969/PUC-RJ-MVP2/main/{nome_arquivo}"

    # Baixa o arquivo usando requests
    resposta = requests.get(url_arquivo, stream=True)
    resposta.raise_for_status()

    # Salva o arquivo no disco
    caminho_arquivo = os.path.join(destino, nome_arquivo)
    with open(caminho_arquivo, 'wb') as arquivo:
        for pedaço in resposta.iter_content(chunk_size=8192):
            arquivo.write(pedaço)

    # Extrai o arquivo ZIP para a pasta de destino
    with zipfile.ZipFile(caminho_arquivo, 'r') as zip_ref:
        zip_ref.extractall(destino)

    # Remove o arquivo .zip
    os.remove(caminho_arquivo)
    print(f"Arquivo '{nome_arquivo}' removido.")

In [ ]:
# Move todos os arquivos para a pasta de destino e remove subpastas, exceto Fotos_Teste e Treinado
dir_excecao = ['Fotos_Teste','Treinado']
for root, dirs, files in os.walk(destino, topdown=False):
    for name in files:
        if not any(exc in root for exc in dir_excecao):   # Ignora Fotos_Teste e treinado
            origem = os.path.join(root, name)
            destino_final = os.path.join(destino, name)

            # Se o arquivo já existe no destino, adiciona um sufixo para evitar conflitos
            if os.path.exists(destino_final):
                base, ext = os.path.splitext(name)
                i = 1
                while os.path.exists(os.path.join(destino, f"{base}_{i}{ext}")):
                    i += 1
                destino_final = os.path.join(destino, f"{base}_{i}{ext}")

            shutil.move(origem, destino_final)

    for name in dirs:
        if name != destino.split('/')[-1] and name != 'Fotos_Teste' and name != 'Treinado' and not root.startswith(os.path.join(destino, 'Treinado')):
            shutil.rmtree(os.path.join(root, name))
            print(f"Pasta '{name}' removida.")

print("Processo concluído. Todos os arquivos extraídos e organizados em 'content/dataset'")

In [ ]:
# 1. Definir os caminhos
dataset_dir = '/content/dataset'  # Diretório principal do dataset
images_dir = os.path.join(dataset_dir, 'images')
labels_dir = os.path.join(dataset_dir, 'labels')

In [ ]:
# 2. Criar as pastas (se não existirem)
os.makedirs(images_dir, exist_ok=True)
os.makedirs(labels_dir, exist_ok=True)
os.makedirs(os.path.join(images_dir, 'train'), exist_ok=True)
os.makedirs(os.path.join(images_dir, 'val'), exist_ok=True)
os.makedirs(os.path.join(labels_dir, 'train'), exist_ok=True)
os.makedirs(os.path.join(labels_dir, 'val'), exist_ok=True)


In [ ]:
# 3. Obter todos os arquivos de imagem e anotação, ignorando Fotos_Teste
all_files = glob.glob(os.path.join(dataset_dir, '**', '*'), recursive=True)
all_images = [f for f in all_files if f.lower().endswith(('.jpg', '.jpeg', '.png')) and 'Fotos_Teste' not in f and 'Treinado' not in f]  # Filtra imagens e ignora Fotos_Teste
all_annotations = [f for f in all_files if f.lower().endswith('.csv') and 'Fotos_Teste' not in f and 'Treinado' not in f]  # Filtra anotações e ignora Fotos_Teste


In [ ]:
# 4. Dividir os dados em treino e validação (80% treino, 20% validação)
num_images = len(all_images)
num_train = int(num_images * 0.8)

train_images = all_images[:num_train]
val_images = all_images[num_train:]
train_annotations = all_annotations[:num_train]
val_annotations = all_annotations[num_train:]

In [ ]:
# 5. Mover os arquivos para as pastas corretas
for image_path in train_images:
    shutil.move(image_path, os.path.join(images_dir, 'train'))
for annotation_path in train_annotations:
    shutil.move(annotation_path, os.path.join(labels_dir, 'train'))
for image_path in val_images:
    shutil.move(image_path, os.path.join(images_dir, 'val'))
for annotation_path in val_annotations:
    shutil.move(annotation_path, os.path.join(labels_dir, 'val'))


In [ ]:
# Renomear os arquivos CSV para TXT
def convert_csv_to_yolo(csv_file, txt_file, classes):
    """Converte um arquivo CSV de anotações para o formato TXT do YOLO.

    Args:
        csv_file: Caminho para o arquivo CSV.
        txt_file: Caminho para o arquivo TXT de saída.
        classes: Lista de nomes das classes.
    """

    df = pd.read_csv(csv_file)
    with open(txt_file, 'w') as f:

        for index, row in df.iterrows():
            # Assumindo que as colunas do CSV são: filename, width, height, class, xmin, ymin, xmax, ymax
            filename = row['filename']
            width = row['width']
            height = row['height']
            class_name = row['class']
            xmin = row['xmin']
            ymin = row['ymin']
            xmax = row['xmax']
            ymax = row['ymax']

            # Calcular as coordenadas normalizadas do centro, largura e altura
            x_center = (xmin + xmax) / (2 * width)
            y_center = (ymin + ymax) / (2 * height)
            bbox_width = (xmax - xmin) / width
            bbox_height = (ymax - ymin) / height

            # Converter classes para uma lista se for um array NumPy
            if isinstance(classes, np.ndarray):
                classes = classes.tolist()  # Converter para lista para usar append

            # Verificar se a classe está na lista e adicioná-la se não estiver
            if class_name not in classes:
                classes.append(class_name)


            # Obter o índice da classe
            class_index = classes.index(class_name)

            # Escrever a linha no formato YOLO
            f.write(f"{class_index} {x_center} {y_center} {bbox_width} {bbox_height}\n")


    return classes  # Retornar a lista classes modificada


In [ ]:
# Obter todos os arquivos CSV
csv_files = glob.glob(os.path.join(labels_dir, '**', '*.csv'), recursive=True)

# Cria a variavel classes vazia
classes = np.array([])

# Iterar pelos arquivos CSV e convertê-los
for csv_file in csv_files:
    # Criar o caminho para o arquivo TXT correspondente
    txt_file = csv_file.replace('.csv', '.txt')#.replace(labels_dir, images_dir)
    txt_file = os.path.splitext(txt_file)[0] + '.txt'

    # Criar a pasta de destino se não existir
    os.makedirs(os.path.dirname(txt_file), exist_ok=True)

    # Converter o arquivo CSV para TXT padrão YOLO e
    # identificar todas as classes (aviões) que estão dentro dos CSV´s
    classes = convert_csv_to_yolo(csv_file, txt_file, classes)

    #Remove os arquivos CSV´s deixando apenas os TXT´s gerados
    os.remove(csv_file)




In [ ]:
# 6. Criar o arquivo data.yaml
classes_string = ', '.join([f'"{c}"' for c in classes])
with open('/content/dataset/data.yaml', mode='w') as f:
    yaml_string='train: /content/dataset/images/train\n'\
              + 'val: /content/dataset/images/val\n'\
              + f'nc: {len(classes)}\n'\
              + f'names: [{classes_string}]'
    f.write(yaml_string)
print(yaml_string)

train: /content/dataset/images/train
val: /content/dataset/images/val
nc: 34
names: ["A10", "KC135", "C130", "F35", "A400M", "Rafale", "B1", "F16", "F15", "F18", "EF2000", "F22", "Tornado", "Su34", "Su25", "B52", "F4", "C2", "C5", "JAS39", "F117", "C17", "V22", "Mirage2000", "CH47", "AV8B", "E7", "C390", "UH60", "U2", "JF17", "AH64", "P3", "B2"]


In [ ]:
# 7. Treinando o Modelo
model = YOLO('yolov8n.pt')

# Definindo os hiperparametros
hip = {
    'epochs': 2,# O ideal seria acima de 100. Já foi realizado um aprendizado com 100 epochs. Os testes com modelo treinado está mais abaixo
    'batch': 16,
    'imgsz': 640,# testar com 416
    'lr0': 0.005,
    'lrf': 0.0005,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'optimizer': 'SGD'}

results = model.train(data='/content/dataset/data.yaml', **hip)


In [ ]:

for path in sorted(glob.glob('/content/runs/detect/train/*.png')):
    image = cv2.imread(path)[:,:,::-1]
    plt.figure(figsize=(20,20))
    plt.imshow(image)
    plt.show()

In [ ]:
for path in sorted(glob.glob('/content/runs/detect/train/*.jpg')):
    image = cv2.imread(path)[:,:,::-1]
    plt.figure(figsize=(20,20))
    plt.imshow(image)
    plt.show()

In [ ]:

# 4. Processar e exibir os resultados
def processar_imagem(results,image,model):
    for r in results:

        # Obter as caixas delimitadoras e rótulos das detecções
        boxes = r.boxes
        xyxy = boxes.xyxy.cpu().numpy()  # Coordenadas das caixas delimitadoras
        conf = boxes.conf.cpu().numpy()  # Confiança das detecções
        cls = boxes.cls.cpu().numpy()  # Classes das detecções

        # Desenhar as caixas delimitadoras na imagem original
        for i in range(len(boxes)):
            x1, y1, x2, y2 = map(int, xyxy[i])
            label = f'{model.names[int(cls[i])]} {conf[i]:.2f}'
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Desenhar retângulo verde
            cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)  # Escrever rótulo

        # 5. Exibir a imagem com as detecções
        plt.figure(figsize=(10, 10))
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))  # Converter BGR para RGB para o Matplotlib
        plt.axis('off')  # Remover eixos
        plt.show()


In [ ]:
# Validar aprendizado com novas figuras

def testar_imagens_jpg(diretorio,model):

  """
  Varre um diretório e testa todos os arquivos JPG encontrados.

  Args:
    diretorio: O caminho para o diretório a ser varrido.
  """

  # 2. Obter lista de todos os arquivos JPG no diretório
  arquivos_jpg = glob.glob(os.path.join(diretorio, '**', '*.jpg'), recursive=True)

  # 2. Iterar sobre os arquivos e realizar o teste
  for arquivo_jpg in arquivos_jpg:
    print(f"Testando imagem: {arquivo_jpg}")

    # 3. Realizar a inferência
    image = cv2.imread(arquivo_jpg)
    results = model(arquivo_jpg)
    processar_imagem(results,image,model)


In [ ]:
# Testar o aprendizado com imagens diferentes

diretorio = '/content/dataset/Fotos_Teste'

# 1. Carregar o modelo treinado
model = YOLO('/content/runs/detect/train/weights/best.pt')

testar_imagens_jpg(diretorio,model)

***ABAIXO, SEGUE O MESMO TESTE COM UM MODELO TREINADO (100 EPOCHs)***

In [ ]:
for path in sorted(glob.glob('/content/dataset/Treinado/*.png')):
    image = cv2.imread(path)[:,:,::-1]
    plt.figure(figsize=(20,20))
    plt.imshow(image)
    plt.show()

In [ ]:
for path in sorted(glob.glob('/content/dataset/Treinado/*.jpg')):
    image = cv2.imread(path)[:,:,::-1]
    plt.figure(figsize=(20,20))
    plt.imshow(image)
    plt.show()

In [ ]:
# Testar o aprendizado com imagens diferentes e com arquivo já Treinado com 100 Epochs

diretorio = '/content/dataset/Fotos_Teste'

# 1. Carregar o modelo treinado
model = YOLO('/content/dataset/Treinado/train/weights/best.pt')

testar_imagens_jpg(diretorio,model)